# Tutorial: how to use giotto-ph

This tutorial shows the basic functionalities and API of `giotto-ph`.

## Introduction

`giotto-ph` is a reworked and **parallelised** version of the Vietoris-Rips algorithm. In short, this algorithms computes the homology groups of the Vietoris-Rips filtration. Such filtration is obtained by progressively enlarging a *scale* parameter $\epsilon$ to build a sequence of simplicial complexes. The simplicial complexes are built based on the intersection of $\epsilon$-radius bubbbles. Given that the simplicial complexes contain more and more elements as the bubbles inflate, there are canonical injective maps between the complex at $\epsilon_1$ and the complex at $\epsilon_2$, with $\epsilon_1 \leq \epsilon_2$ (hence the ffiltration is well defined).

In this picture we see the sequence of complexes forming while $\epsilon$ increases over time. The bubbles are the disks of radius $\epsilon$.


In [ ]:
from IPython.display import Image  # to display images
Image("images/ph.gif")

## A word on parallelism

The parallelism of `giotto-ph` builds on different sources already present in literature: for example, you can have a look [here](https://arxiv.org/abs/2003.07989) and [here](https://www.mrzv.org/publications/lockfree-persistence/spaa/).

The basic idea is to parallelise the boundary matrix $\bf D$ reduction step. Indeed, in order to compute homology, one needs to know the *rank* and the *kernel* of the boundary martrix $\bf D$ ([here](https://en.wikipedia.org/wiki/Simplicial_homology) for more details). In order to easily read the kernel and the rank from $\bf D$ (and be able to quotient them to get the homology), the most effective approach is to reduce $\bf D$ in *echelon form*: the kernel is then the number of columns of all zeros and the rank corresponds to the non-empty rows. In order to reduce the orginal $\bf D$ to echelon form, we can add columns to one another (from left to right) till convergence: this step does not require a precise order in the operations: it requires only to do the correct operations! Hence, the possibility to parallelise it.

In [ ]:
# Install missing dependencies
import sys
!{sys.executable} -m pip install giotto-tda

In [ ]:
# here comes our protagonist!
from gph import ripser_parallel

# Import utils
import numpy as np
from gtda.homology._utils import _postprocess_diagrams

# To generate dataset
from sklearn import datasets

# Plotting
from plotly import graph_objects as go
from gtda.plotting import plot_diagram, plot_point_cloud

In [ ]:
# build the point cloud
data = datasets.make_circles(n_samples=100)[0] + 5 * datasets.make_circles(n_samples=100)[0]

# plot the point cloud
plot_point_cloud(data)

## Default arguments

Using default arguments is possible and easy: unfortunately it may take some time as you will not unlock the parallised computations...

### A word on persistence diagrams
One of the most effective way of plotting the persistent topological features is to plot them in the plane $(b, d)$ whose axes are the birth and death filtration values, i.e. the values where the topological feature is created (e.g. the circle is formed) and when the feature is destroyed (e.g. a circle is filled up). Hence, the more the points are far from the diagonal $b = d$, the more a topological feature is *persistent*.

In [ ]:
# compute the persistence diagram
dgm = ripser_parallel(data)

# convert to gtda format
dgm_gtda = _postprocess_diagrams([dgm["dgms"]], "ripser", (0, 1), np.inf, True)[0]

# plot the persistence diagram
plot_diagram(dgm_gtda, homology_dimensions=(0, 1))

## Changing the basis field

It is possible to change the basis field of the homology group (actually, a vector space in this case!), to any finite filed in (prime) characteristic.

The default is $\mathbb F_2$, but it can be generalised to any $\mathbb F_p$, $p$ prime.

In [ ]:
# compute the persistence diagram
dgm = ripser_parallel(data, coeff=7)

# convert to gtda format
dgm_gtda = _postprocess_diagrams([dgm["dgms"]], "ripser", (0, 1), np.inf, True)[0]

# plot
plot_diagram(dgm_gtda, homology_dimensions=(0, 1))

# Higher homology groups
We can compute any order of homology, $H_0$,$H_1$,$H_2$,…

By default, we only compute $H_0$ and $H_1$. 

You can specify a larger group by setting the argument `maxdim=p`. It practice, anything above $H_1$ will benefit more substantially form parallelisation.

In [ ]:
# compute the persistence diagram
dgm = ripser_parallel(data, maxdim=2)

# comnvert to gtda format
dgm_gtda = _postprocess_diagrams([dgm["dgms"]], "ripser", (0, 1, 2), np.inf, True)[0]

# plot
plot_diagram(dgm_gtda, homology_dimensions=(0, 1, 2))

## Specify a maximum radius

We can decide what is the maximum size of the bubbles and stop the computations there. With less simplices to compute, the computations will be faster if you specify such thresholds. In order to do so, please add the argument `thresh=2.5`.

#### Warning
Reducing the threshold implies that topological features that only appear at a large radius (a.k.a. filtration value) may not be present at all in your persistence diagram or that some features may not die (as in the example below).

In [ ]:
# compute the persistence diagram
dgm = ripser_parallel(data, thresh=2.5)

# convert to gtda format
dgm_gtda = _postprocess_diagrams([dgm["dgms"]], "ripser", (0, 1), np.inf, True)[0]

# plot
plot_diagram(dgm_gtda, homology_dimensions=(0, 1))

## Edge Collapser integration

By setting the optional parameter `collapse_edges` to `True`, the [Edge Collapse](https://hal.inria.fr/hal-02873740/document) algorithm is used before performing any matrix reduction. This algorithm flags some of the edges as *dominated* and removes them completely from the filtration. This can lead to a greatly sparsified filtration and therefore to immense speed-ups especially when high homology dimensions are required.

**Persistent barcodes computed with or without edge collapses are exactly the same**.

In [ ]:
# compute the persistence diagram
dgm = ripser_parallel(data, collapse_edges=True)

# convert to gtda format
dgm_gtda = _postprocess_diagrams([dgm["dgms"]], "ripser", (0, 1), np.inf, True)[0]

# plot
plot_diagram(dgm_gtda, homology_dimensions=(0, 1))

## Retrieve flag persistence generators

You can retrieve the vertices and edges (pairs of vertex indices) responsible for the creation/destruction of persistent topological features by passing `return_generators=True`. A new entry is added to the dictionary output by `ripser_parallel`, with key `"gens"` and corresponding value a tuple of length 4 organized schematically as follows:

  0. vertices creating and edges destroying *finite* $0$-dimensional features;
  1. edges creating and destroying *finite* $d$-dimensional features, $d \geq 1$;
  2. vertices creating *infinite* $0$-dimensional features;
  3. edges creating *infinite* $d$-dimensional features, $d \geq 1$.

In the case of entries 1 and 3 (higher dimensions), that information is organized by homology dimension. So, for example, calling `gens` this tuple (value in the dictionary):

  - `gens[1]` and `gens[3]` are lists containing `maxdim` 2D integer `numpy` arrays, while `gens[0]` and `gens[2]` are `numpy` arrays;
  - the edges creating and destroying finite features in dimension 1 are stored in `gens_finite_1 = gens[1][0]`;
  - `gens_finite_1` is a 2D integer `numpy` array with as many rows as there are finite features in dimension 1;
  - The `i`th finite feature in the 1-dimensional barcode is created by edge `gens_finite_1[i, :2]` and destroyed by edge `gens_finite_1[i, 2:]`.

This way of presenting persistence birth and death vertices/edges agrees with other persistent homology packages and in particular with [GUDHI](http://gudhi.gforge.inria.fr/).

In [ ]:
# Compute the persistence information
persistence_info = ripser_parallel(data, return_generators=True)
gens = persistence_info['gens']

Let us visualize the 1-dimensional generators for our point cloud, labelling them by the positional index of the corresponding feature in the persistence diagram and using blue for creation and red for destruction:

In [ ]:
# Plot the point cloud
fig = plot_point_cloud(data)

# In blue are the edges that create a finite persistent topological features in dimension 1.
# In red are the edges that destroy a finite persistent topological feature in dimension 1.
for i, edges in enumerate(gens[1][0]):
    birth_edge = edges[0:2]
    death_edge = edges[2:4]
    x0_create, y0_create = data[birth_edge[0]]
    x1_create, y1_create = data[birth_edge[1]]
    x0_destroy, y0_destroy = data[death_edge[0]]
    x1_destroy, y1_destroy  = data[death_edge[1]]  

    fig.add_shape(type='line',
                  x0=x0_create,
                  y0=y0_create,
                  x1=x1_create,
                  y1=y1_create,
                  line=dict(color='Blue'))
    fig.add_shape(type='line',
                  x0=x0_destroy,
                  y0=y0_destroy,
                  x1=x1_destroy,
                  y1=y1_destroy,
                  line=dict(color='Red'))
    fig.add_trace(
        go.Scatter(x=[0.5 * (x0_create + x1_create) + 0.2, 0.5 * (x0_destroy + x1_destroy) + 0.2],
                   y=[0.5 * (y0_create + y1_create), 0.5 * (y0_destroy + y1_destroy)],
                   text=[str(i), str(i)],
                   mode="text")
    )

fig.update_layout(showlegend=False)
fig.show()

## Unleash multiple threads

By changing the parameter `n_threads=N`, with `N >= -1`, you can parallelise the computation of the matrix reduction. This parallelisation is different from the `n_jobs` one that you can use on [giotto-tda](https://giotto-ai.github.io/gtda-docs/0.4.0/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html#gtda.homology.VietorisRipsPersistence): the latter parallelises over multiple point clouds, whereas the former better distributes the computations for a single point cloud. With powerful enough machines, you can, of course, exploit both of the parallelisations!


In [ ]:
Image("images/multithread.png")

In [ ]:
# compute the persistence diagram
dgm = ripser_parallel(data, n_threads=8, maxdim=5)

# convert to gtda format
dgm_gtda = _postprocess_diagrams([dgm["dgms"]], "ripser", (0, 1, 2, 3, 4, 5), np.inf, True)[0]

# plot
plot_diagram(dgm_gtda, homology_dimensions=(0, 1, 2, 3, 4, 5))